In [3]:
import os

train_done = True
study_id = 'original'
data_id = 'pan'

size = '100p'
per = 'all'

data_ = 'TransMut'
encoding_ = 'embedding'

id_ = 'OF_TData_pad14_{}_onlyEmbedding_NOPOS'.format(size)

data_dir = '/home/s202357/thesis/transmut/data/transmut_github/'
model_folder = '/home/s202357/thesis/transmut/pipeline/{}/models/'.format(study_id)

result_folder = '/home/s202357/thesis/transmut/pipeline/{}/results/'.format(study_id)

plot_name = result_folder + id_+ '_plot_'
df_name = result_folder + id_+ '_df_'

if not os.path.exists(model_folder):
    print('creating dirs')
    os.makedirs(model_folder)
    os.makedirs(result_folder)
else:
    print('dirs exist')


dirs exist


In [ ]:
!gpustat

In [3]:
os.environ['CUDA_VISIBLE_DEVICES']='1'

In [4]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
import gc
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

#import framework as fr

In [5]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    use_cuda = True
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) avalable.


In [6]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [7]:
def best_treshold(df_results):
    acc_best = -999
    best_i = -999

    for i in np.arange(0, 1, 0.1):
        df_results['pred_binary_'] = np.where((df_results.pred_ > i) , 1, 0)
        acc = accuracy_score(df_results['target_'], df_results['pred_binary_'])
        if acc > acc_best:
            df = df_results
            best_i = i
            acc_best = acc
            
    return acc_best, best_i, df


def best_treshold(df_results, t= []):
    acc_best = -999
    best_i = -999

    if t == []:
        for i in np.arange(0, 1, 0.1):
            df_results['pred_binary_'] = np.where((df_results.pred_ > i) , 1, 0)
            acc = accuracy_score(df_results['target_'], df_results['pred_binary_'])
            if acc > acc_best:
                df = df_results
                best_i = i
                acc_best = acc
    else:
        best_i = t[0]
        df_results['pred_binary'] = np.where((df_results.pred > t[0]) , 1, 0)
        df = df_results
    return acc_best, best_i, df

def pkl(id_, n_layers, n_heads, model_num, d_model):
    return '{}_d{}_layer{}_multihead{}_MODEL{}.pkl'.format(id_, d_model, n_layers, n_heads, model_num)


def make_data(data):
    pep_inputs, hla_inputs, labels = [], [], []
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        pep, hla = pep.ljust(pep_max_len, '-'), hla.ljust(hla_max_len, '-')
        pep_input = [[vocab[n] for n in pep]] # [[1, 2, 3, 4, 0], [1, 2, 3, 5, 0]]
        hla_input = [[vocab[n] for n in hla]]
        pep_inputs.extend(pep_input)
        hla_inputs.extend(hla_input)
        labels.append(label)
    return torch.LongTensor(pep_inputs), torch.LongTensor(hla_inputs), torch.LongTensor(labels)

class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self):
        return self.pep_inputs.shape[0]

    def __getitem__(self, idx):
        return self.pep_inputs[idx], self.hla_inputs[idx], self.labels[idx]

In [8]:

def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q = seq_q.size()
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.src_emb = nn.Embedding(vocab_size, d_model)
        #self.pos_emb = PositionalEncoding(d_model)
        #self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        '''
        enc_inputs: [batch_size, src_len]
        '''
        enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        
        #enc_outputs = self.pos_emb(enc_outputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        
        return enc_outputs, 0


class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        #self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        #print("\n\nAfter concat:", enc_outputs, "\nsize", np.shape(enc_outputs))
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        #dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = enc_outputs.view(enc_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        #print("len(dec_outputs), len(dec_outputs[0])", len(dec_outputs), len(dec_outputs[0]))
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), 0, 0, 0

In [9]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)



def performances_QM_(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)



def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


f_mean = lambda l: sum(l)/len(l)


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [10]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list, dec_attns_train_list = [], []
    for train_pep_inputs, train_hla_inputs, train_labels in train_loader:
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        train_pep_inputs, train_hla_inputs, train_labels = train_pep_inputs.to(device), train_hla_inputs.to(device), train_labels.to(device)

        t1 = time.time()
        train_outputs, _, _, train_dec_self_attns = model(train_pep_inputs, train_hla_inputs)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss)
#         dec_attns_train_list.append(train_dec_self_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, dec_attns_train_list


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

In [11]:
pep_max_len = 14 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load( data_dir + 'Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

# Transformer Parameters
d_model = 64  # Embedding Size
d_k = d_v = 64  # dimension of K(=Q), V

d_ff = 512 # FeedForward dimension
n_layers = 1  # number of Encoder of Decoder Layer
n_heads = 3

batch_size = 1024
epochs = 25
threshold = 0.5

In [12]:
###### -> upd
#index_order_nested

cv = 5
count = 0
index_order = []

for outer in range(cv):
        #print(f"\nk_outer: {outer}")
        #print("test: " + str(outer))
        test_idx = outer
        for inner in range(cv):
            #print(f"\t\nk_inner: {inner}")
            if inner!=outer:
                #print("\tval: " + str(inner))
                val_idx = inner
                train_index = list()
                for t in range(5):
                    if t!=inner and t!=outer:
                        train_index.append(t)
                #print("\ttrain:", str(train_index))
                count += 1  
                index_order.append([test_idx, val_idx, train_index])
                
index_order_nested = index_order


###### -> upd - nested

def data_with_loader(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df_list.append(pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0)) #[0:per]
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[index_order_nested[index_order_idx][0]]
        print("TEST ID:", index_order_nested[index_order_idx][0], len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Fold-{} Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list, data
    gc.collect()
    return pep_inputs, hla_inputs, labels, loader

index_order = 0

In [18]:
# continue here 

ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list, test_fold_metrics_list = [], [], []
#independent_fold_metrics_list, external_fold_metrics_list, ys_independent_fold_dict, ys_external_fold_dict = [], [], {}, {}
attns_train_fold_dict, attns_val_fold_dict = {}, {}
loss_train_fold_dict, loss_val_fold_dict, = {}, {}
metrics_all_val = [] 
metrics_all_train = [] 

time_train_c = time.time()

for fold in tqdm(range(20)): #####!!!!!!!

        print('=====Inner Fold-{}====='.format(fold))
        print('-----Generate data loader-----')
        index_order_idx = fold
        train_pep_inputs, train_hla_inputs, train_labels, train_loader = data_with_loader(index_order_idx, type_ = 'train', fold = fold,  batch_size = batch_size)
        val_pep_inputs, val_hla_inputs, val_labels, val_loader = data_with_loader(index_order_idx, type_ = 'val', fold = fold,  batch_size = batch_size)

        print('TEST ID', index_order_nested[index_order_idx][0])

        print('-----Compile model-----')
        model = Transformer().to(device)
        criterion = nn.CrossEntropyLoss() 
        optimizer = optim.Adam(model.parameters(), lr = 1e-3) #, momentum = 0.99)

        print('-----Train-----')
        path_saver = model_folder + pkl(id_, n_layers, n_heads, fold, d_model)

        print('path_saver: ', path_saver)

        metric_best, ep_best = 0, -1
        time_train = 0

        for epoch in range(1, epochs + 1):
            time_train_f = time.time()
            ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
            ys_val, loss_val_list, metrics_val, _prob, _binary = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

            train_fold_metrics_list.append(metrics_train)
            val_fold_metrics_list.append(metrics_val)

            metrics_ep_avg = sum(metrics_val[:4])/4
            if metrics_ep_avg > metric_best: 
                metric_best, ep_best = metrics_ep_avg, epoch
                torch.save(model.eval().state_dict(), path_saver)
            print("Fold time: {:6.2f} mins.".format(   round( (time.time()-time_train_f)/60, 2)   ))

        print("best_epoch:", ep_best)
        print('-----Optimization Finished!-----')
print("Total training time: {:6.2f} mins.".format(   round( (time.time()-time_train_c)/60, 2)   ))

  0%|                                                                                                             | 0/20 [00:00<?, ?it/s]

=====Inner Fold-0=====
-----Generate data loader-----
TRAIN ID: [2, 3, 4] 1811042
Fold-0 Label info: Data train = Counter({1: 905928, 0: 905114}) 
VAL ID: 1 602231
Fold-0 Label info: Data val = Counter({0: 301884, 1: 300347}) 
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_NOPOS_d32_layer1_multihead3_MODEL0.pkl
Fold-0****Train (Ep avg): Epoch-1/25 | Loss = 0.2211 | Time = 1.1029 sec
tn = 822280, fp = 82834, fn = 77640, tp = 828288
y_pred: 0 = 899920 | 1 = 911122
y_true: 0 = 905114 | 1 = 905928
auc=0.9701|sensitivity=0.9143|specificity=0.9085|acc=0.9114|mcc=0.8228
precision=0.9091|recall=0.9143|f1=0.9117|aupr=0.9687
Fold-0 ****Test  Epoch-1/25: Loss = 0.208756
tn = 280670, fp = 21214, fn = 27844, tp = 272503
y_pred: 0 = 308514 | 1 = 293717
y_true: 0 = 301884 | 1 = 300347
auc=0.9737|sensitivity=0.9073|specificity=0.9297|acc=0.9185|mcc=0.8373
precision=0.9278|recall=0.9073|f1=0.9174|au

Fold-0****Train (Ep avg): Epoch-14/25 | Loss = 0.1116 | Time = 1.0660 sec
tn = 861982, fp = 43132, fn = 30366, tp = 875562
y_pred: 0 = 892348 | 1 = 918694
y_true: 0 = 905114 | 1 = 905928
auc=0.9915|sensitivity=0.9665|specificity=0.9523|acc=0.9594|mcc=0.9189
precision=0.9531|recall=0.9665|f1=0.9597|aupr=0.9900
Fold-0 ****Test  Epoch-14/25: Loss = 0.264576
tn = 285131, fp = 16753, fn = 33417, tp = 266930
y_pred: 0 = 318548 | 1 = 283683
y_true: 0 = 301884 | 1 = 300347
auc=0.9734|sensitivity=0.8887|specificity=0.9445|acc=0.9167|mcc=0.8346
precision=0.9409|recall=0.8887|f1=0.9141|aupr=0.9734
Fold time:   0.64 mins.
Fold-0****Train (Ep avg): Epoch-15/25 | Loss = 0.1088 | Time = 1.0844 sec
tn = 862963, fp = 42151, fn = 29200, tp = 876728
y_pred: 0 = 892163 | 1 = 918879
y_true: 0 = 905114 | 1 = 905928
auc=0.9918|sensitivity=0.9678|specificity=0.9534|acc=0.9606|mcc=0.9213
precision=0.9541|recall=0.9678|f1=0.9609|aupr=0.9904
Fold-0 ****Test  Epoch-15/25: Loss = 0.271099
tn = 285278, fp = 16606, 

  5%|████▊                                                                                            | 1/20 [16:50<5:20:01, 1010.62s/it]

tn = 285809, fp = 16075, fn = 37742, tp = 262605
y_pred: 0 = 323551 | 1 = 278680
y_true: 0 = 301884 | 1 = 300347
auc=0.9707|sensitivity=0.8743|specificity=0.9468|acc=0.9106|mcc=0.8234
precision=0.9423|recall=0.8743|f1=0.9071|aupr=0.9705
Fold time:   0.68 mins.
best_epoch: 4
-----Optimization Finished!-----
=====Inner Fold-1=====
-----Generate data loader-----
TRAIN ID: [1, 3, 4] 1809917
Fold-1 Label info: Data train = Counter({1: 905010, 0: 904907}) 
VAL ID: 2 603356
Fold-1 Label info: Data val = Counter({0: 302091, 1: 301265}) 
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_NOPOS_d32_layer1_multihead3_MODEL1.pkl
Fold-1****Train (Ep avg): Epoch-1/25 | Loss = 0.2189 | Time = 1.0851 sec
tn = 823129, fp = 81778, fn = 76860, tp = 828150
y_pred: 0 = 899989 | 1 = 909928
y_true: 0 = 904907 | 1 = 905010
auc=0.9707|sensitivity=0.9151|specificity=0.9096|acc=0.9124|mcc=0.8247
precision=0.9101|

...

 50%|███████████████████████████████████████████████                                               | 10/20 [2:58:05<3:00:00, 1080.02s/it]

tn = 284541, fp = 17343, fn = 35132, tp = 265215
y_pred: 0 = 319673 | 1 = 282558
y_true: 0 = 301884 | 1 = 300347
auc=0.9708|sensitivity=0.8830|specificity=0.9426|acc=0.9129|mcc=0.8271
precision=0.9386|recall=0.8830|f1=0.9100|aupr=0.9708
Fold time:   0.74 mins.
best_epoch: 3
-----Optimization Finished!-----
=====Inner Fold-10=====
-----Generate data loader-----
TRAIN ID: [0, 1, 4] 1811332
Fold-10 Label info: Data train = Counter({1: 906421, 0: 904911}) 
VAL ID: 3 602314
Fold-10 Label info: Data val = Counter({0: 301499, 1: 300815}) 
TEST ID 2
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_NOPOS_d32_layer1_multihead3_MODEL10.pkl
Fold-10****Train (Ep avg): Epoch-1/25 | Loss = 0.2190 | Time = 1.1746 sec
tn = 822979, fp = 81932, fn = 77002, tp = 829419
y_pred: 0 = 899981 | 1 = 911351
y_true: 0 = 904911 | 1 = 906421
auc=0.9707|sensitivity=0.9150|specificity=0.9095|acc=0.9123|mcc=0.8245
precision=0.

tn = 861331, fp = 43580, fn = 30638, tp = 875783
y_pred: 0 = 891969 | 1 = 919363
y_true: 0 = 904911 | 1 = 906421
auc=0.9914|sensitivity=0.9662|specificity=0.9518|acc=0.9590|mcc=0.9181
precision=0.9526|recall=0.9662|f1=0.9594|aupr=0.9900
Fold-10 ****Test  Epoch-13/25: Loss = 0.260436
tn = 284502, fp = 16997, fn = 33109, tp = 267706
y_pred: 0 = 317611 | 1 = 284703
y_true: 0 = 301499 | 1 = 300815
auc=0.9734|sensitivity=0.8899|specificity=0.9436|acc=0.9168|mcc=0.8348
precision=0.9403|recall=0.8899|f1=0.9144|aupr=0.9733
Fold time:   0.72 mins.
Fold-10****Train (Ep avg): Epoch-14/25 | Loss = 0.1087 | Time = 1.1491 sec
tn = 862550, fp = 42361, fn = 29128, tp = 877293
y_pred: 0 = 891678 | 1 = 919654
y_true: 0 = 904911 | 1 = 906421
auc=0.9918|sensitivity=0.9679|specificity=0.9532|acc=0.9605|mcc=0.9212
precision=0.9539|recall=0.9679|f1=0.9609|aupr=0.9904
Fold-10 ****Test  Epoch-14/25: Loss = 0.267969
tn = 284568, fp = 16931, fn = 33728, tp = 267087
y_pred: 0 = 318296 | 1 = 284018
y_true: 0 = 301

 50%|███████████████████████████████████████████████                                               | 10/20 [3:10:12<3:10:12, 1141.26s/it]


KeyboardInterrupt: 

In [14]:
def data_with_loader_unique(index_order_idx, type_ = 'train', fold = None,  batch_size = 1024):
    df_list = []
    for i in range(5):
        df = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0).drop_duplicates()
        df_list.append(df)
        
    if type_ != 'train' and type_ != 'val' and type_ != 'test':
        data = pd.read_csv(data_dir + '{}_set.csv'.format(type_), index_col = 0)
    elif type_ == 'train':
        data = pd.concat([ df_list[index_order_nested[index_order_idx][2][0]], df_list[index_order_nested[index_order_idx][2][1]], df_list[index_order_nested[index_order_idx][2][2]] ]).reset_index().drop(["index"], axis=1)
        print("TRAIN ID:", index_order_nested[index_order_idx][2], len(data))
    elif type_ == 'test':
        data = df_list[index_order_nested[index_order_idx][0]]
        print("TEST ID:", index_order_nested[index_order_idx][0], len(data))
    elif type_ == 'val':
        data = df_list[index_order_nested[index_order_idx][1]]
        print("VAL ID:", index_order_nested[index_order_idx][1], len(data))
        
    pep_inputs, hla_inputs, labels = make_data(data)
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    print('Label info: Data {} = {} '.format(fold, type_, Counter(data.label)))
    
    del df_list
    gc.collect()
    return data, pep_inputs, hla_inputs, labels, loader

def eval_step_test(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)
            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print_bool = False
        if epoch == 1 or epoch == 25 or epoch == 50:
            print_bool = True
            print('\nFold-{} **** Test  Epoch-{}/{}: Loss = {:.6f} | '.format(fold, epoch, epochs, f_mean(loss_val_list)), end='')
            
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list, print_ = print_bool)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

def binary_roc_auc_score(y_true, y_score, sample_weight=None, max_fpr=None):
    """Binary roc auc score."""
    if len(np.unique(y_true)) != 2:
        raise ValueError(
            "Only one class present in y_true. ROC AUC score "
            "is not defined in that case.")

    fpr, tpr, _ = roc_curve(y_true, y_score, sample_weight=sample_weight)
    if max_fpr is None or max_fpr == 1:
        return auc(fpr, tpr)
    if max_fpr <= 0 or max_fpr > 1:
        raise ValueError("Expected max_fpr in range (0, 1], got: %r" % max_fpr)

    # Add a single point at max_fpr by linear interpolation
    stop = np.searchsorted(fpr, max_fpr, "right")
    x_interp = [fpr[stop - 1], fpr[stop]]
    y_interp = [tpr[stop - 1], tpr[stop]]
    tpr = np.append(tpr[:stop], np.interp(max_fpr, x_interp, y_interp))
    fpr = np.append(fpr[:stop], max_fpr)
    partial_auc = auc(fpr, tpr)
    return partial_auc/0.1


In [15]:
id_nested = id_

ep_best = 0
criterion = nn.CrossEntropyLoss()

test_fold_metrics_list_nested = []

fold_avg_4 = True
type_ = 'test'

df_list = []
time_testing = time.time()

if fold_avg_4:
    
    fold_avg = [list(range(0,4)), list(range(4,8)), list(range(8,12)), list(range(12,16)), list(range(16,20))] 
    pred_all = []
    target_all = []

    for fold in fold_avg:

            pred_fold = []
            target_fold = []
            file_num = fold[0]
            print('\n=====Models-{}====='.format(fold))
            
            for f in fold:
                index_order_idx = f
                test_data, test_pep_inputs, test_hla_inputs, test_labels, test_loader = data_with_loader_unique(index_order_idx, type_ = type_, fold = file_num,  batch_size = batch_size)
                model = Transformer().to(device)
                path_saver = model_folder + pkl(id_nested, n_layers, n_heads, f, d_model)
                print(path_saver)
                #
                model.load_state_dict(torch.load(path_saver))
                model_test = model.eval()

                ys_res_test, loss_res_test_list, metrics_res_test, y_prob, y_pred_bin = eval_step_test(model_test, test_loader, fold, ep_best, epochs, use_cuda) # , test_res_attns
                pred_fold.append(y_prob)
                target_fold = [x.item() for x in test_labels]
                
                ## add auc01 and ppv
                df_temp = pd.DataFrame(list(zip(target_fold, y_prob)), columns= ['target_', 'pred_'])
                num_pos = len(df_temp[df_temp['target_']==1])
                df_temp_ppv = df_temp.sort_values(by=['pred_'], ascending=False)[0:num_pos]
                num_true_pos = len(df_temp_ppv[df_temp_ppv['target_']==1])
                ppv = num_true_pos/num_pos
                auc_01 = binary_roc_auc_score(df_temp['target_'].tolist(), df_temp['pred_'], max_fpr=0.1)
                auc01_ppv = (auc_01, ppv)
                metrics_final = (*auc01_ppv, *metrics_res_test)
                
                test_fold_metrics_list_nested.append(metrics_final)
                
                print("\t\t\t\t roc in:", roc_auc_score(target_fold, y_prob))

            # get avg of pred_fold and extend pred_all
            arrays = [np.array(x) for x in pred_fold]
            pred_fold_avg = [np.mean(k) for k in zip(*arrays)]
            pred_all.extend(pred_fold_avg)
            target_all.extend(target_fold)
            print("\t\t roc avg accmmltd:", roc_auc_score(target_all, pred_all))  
            
            test_data['target_'] = target_fold
            test_data['pred_'] = pred_fold_avg
            df_list.append(test_data)


    df_pd_nested_test_unique = pd.concat(df_list)
    del df_list
    gc.collect()
    
print("Time testing:", round((time.time()-time_testing)/60, 2), "mins. \n\n")


=====Models-[0, 1, 2, 3]=====
TEST ID: 0 372896
Label info: Data 0 = test 
/home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_d32_layer1_multihead3_MODEL0.pkl
				 roc in: 0.9781974967046836
TEST ID: 0 372896
Label info: Data 0 = test 
/home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_d32_layer1_multihead3_MODEL1.pkl
				 roc in: 0.9775966422887225
TEST ID: 0 372896
Label info: Data 0 = test 
/home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_d32_layer1_multihead3_MODEL2.pkl
				 roc in: 0.9782443249692359
TEST ID: 0 372896
Label info: Data 0 = test 
/home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_100p_onlyEmbedding_d32_layer1_multihead3_MODEL3.pkl
				 roc in: 0.9772686557374799
		 roc avg accmmltd: 0.9812494219797439

=====Models-[4, 5, 6, 7]=====
TEST ID: 1 372782
Label info: Data 4 = test 
/home/s202357/thesis/transmut/pipeline/original/models/OF

In [16]:
df_results = pd.DataFrame(list(zip(target_all, pred_all)), columns= ['target', 'pred'])
print("\nSTATS:")
auc_ = roc_auc_score(df_results['target'], df_results['pred'])
print("AUC:", auc_)


num_pos = len(df_results[df_results['target']==1])
df_temp_ppv = df_results.sort_values(by=['pred'], ascending=False)[0:num_pos]
num_true_pos = len(df_temp_ppv[df_temp_ppv['target']==1])
ppv = num_true_pos/num_pos
auc_01 = binary_roc_auc_score(df_results['target'].tolist(), df_results['pred'].tolist(), max_fpr=0.1)
print("AUC FPR 0.1:", auc_01)
print("PPV:", ppv)


print("\nPerformance adv:")

acc_best, i, df_results = best_treshold(df_results, [0.5])
print("Used/Best treshold:", i, "\n")
print("AUC pred treshold:", roc_auc_score(df_results['target'], df_results['pred_binary']))
roc_auc, acc, mcc, f1, sensitivity, specificity, precision, recall, aupr = performances(df_results['target'], df_results['pred_binary'], df_results['pred'], print_ = True)

#df_pd_nested_test_unique.to_csv(df_name + 'test_avg4.csv', index=False)



STATS:
AUC: 0.9805988919210391
AUC FPR 0.1: 0.8638211899603461
PPV: 0.8700740047777351

Performance adv:
Used/Best treshold: 0.5 

AUC pred treshold: 0.9323538280498724
tn = 1412589, fp = 92865, fn = 26437, tp = 332729
y_pred: 0 = 1439026 | 1 = 425594
y_true: 0 = 1505454 | 1 = 359166
auc=0.9806|sensitivity=0.9264|specificity=0.9383|acc=0.9360|mcc=0.8125
precision=0.7818|recall=0.9264|f1=0.8480|aupr=0.9355


In [17]:
attr = ['id', 'data', 'data_size', 'data_info', 'encoding', 'results_df', 'pep_max_len', 'hla_max_len', 'tgt_len', 'd_model', 'd_k', 'd_v', 'd_ff', 'n_layers', 'n_heads', 'batch_size', 'epochs', 'treshold', 'test_dup', 'test_type', 'auc', 'auc01', 'ppv', 'acc', 'mcc', 'sensitivity', 'specificity', 'precision', 'recall', 'f1', 'aupr']

test_dup = 'tested on unique'

global_info = [id_, data_ , size, per, encoding_ , df_name + 'test_avg4.csv', pep_max_len, hla_max_len, tgt_len, d_model, d_k, d_v, d_ff, n_layers, n_heads, batch_size, epochs, threshold, test_dup, 'average of each 4 - then concat', auc_, auc_01, ppv, acc, mcc, sensitivity, specificity, precision, recall, f1, aupr]

for i,t in zip(global_info, attr):
    print(t, ":", i)
    
with open(result_folder + 'results_global', 'a') as file:
    file.write('\n'+','.join([str(x) for x in global_info]))
file.close()

id : OF_TData_pad14_100p_onlyEmbedding
data : TransMut
data_size : 100p
data_info : all
encoding : embedding
results_df : /home/s202357/thesis/transmut/pipeline/original/results/OF_TData_pad14_100p_onlyEmbedding_df_test_avg4.csv
pep_max_len : 14
hla_max_len : 34
tgt_len : 48
d_model : 32
d_k : 32
d_v : 32
d_ff : 512
n_layers : 1
n_heads : 3
batch_size : 1024
epochs : 25
treshold : 0.5
test_dup : tested on unique
test_type : average of each 4 - then concat
auc : 0.9805988919210391
auc01 : 0.8638211899603461
ppv : 0.8700740047777351
acc : 0.9360180626615611
mcc : 0.8124910570744778
sensitivity : 0.9263933668554373
specificity : 0.9383142892443077
precision : 0.7817990855134236
recall : 0.9263933668554373
f1 : 0.8479764513991538
aupr : 0.9354777898488882
